<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/functionalcpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ
1. C++で関数型プログラミング風に書きたい、と思った。
1. functional programming in C++ という本のchapter01がネットで読めるので、導入だけでもいいと思って読んでいるところ。
1. https://itbook.store/files/9781617293818/chapter1.pdf

# 関数型プログラミング入門

In [ ]:
# 最初にでてくる、関数型でない書き方の例を、とりあえず動かす。
%%writefile countlines01.cpp
#include <iostream>
#include <fstream>
#include <vector>
std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        std::ifstream in(file);
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}
int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines01.cpp


In [ ]:
!g++ countlines01.cpp -o countlines01; ./countlines01

29


In [ ]:
!wc countlines01.cpp

 29  75 702 countlines01.cpp


行数が一致することを確認。

In [ ]:
# std::count アルゴリズムを使う
%%writefile countlines02.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}
std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results;
    for (const auto& file : files) {
        results.push_back(count_lines(file));
    }
    return results;
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines02.cpp


In [ ]:
!g++ countlines02.cpp -o countlines02; ./countlines02

29


In [ ]:
%%writefile countlines03.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results(files.size());
    std::transform(files.cbegin(), files.cend(), results.begin(), count_lines);
    return results;
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines03.cpp


In [ ]:
!g++ countlines03.cpp -o countlines03; ./countlines03

29


In [ ]:
# 次のコードはRangesを使っていて、いまの段階では動かない。
%%script false
%%writefile countlines04.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    return files | transform(count_lines);
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


In [ ]:
# 次のコードはRangesを使っていて、いまの段階では動かない。
%%script false
%%writefile countlines05.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    return files | transform(open_file) | transform(count_lines);
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


In [ ]:
!g++ countlines05.cpp -o countlines05; ./countlines05

In [ ]:
#実験
%%script false
%%writefile transform01.cpp
#include <algorithm>
#include <cctype>
#include <functional>
#include <iostream>
#include <string>
#include <vector>

int main() {
    std::string s("hello");

    namespace ranges = std::ranges;

    ranges::transform(
        s.begin(), s.end(), s.begin(),
        [](unsigned char c) -> unsigned char { return std::toupper(c); });

    std::vector<std::size_t> ordinals;
    ranges::transform(s, std::back_inserter(ordinals),
                      [](unsigned char c) -> std::size_t { return c; });

    std::cout << s << ':';
    for (auto ord : ordinals) {
        std::cout << ' ' << ord;
    }

    ranges::transform(ordinals, ordinals, ordinals.begin(), std::plus{});

    std::cout << '\n';
    for (auto ord : ordinals) {
        std::cout << ord << ' ';
    }
    std::cout << '\n';
}

Writing transform01.cpp


## この本のまとめ、感想
目次を見るとC++でモナドの話まで持っていこうとしている。 それはそれだが、わたしが目的としているのはそんな高度な話ではなくて、
1. for構文がイヤ。 やたらに出てくる。 これを vector で対応できないか。
1. c++20 の range を使いたいわけではない。 #include <arlgrithm> で使える transform() を使いたい。
ということは、vectorとtransformを勉強しよう。

# C++11でcopy_ifと無名関数が使えるようになったので改めて
という記事があって、これをなぞってみる。

# reduce() みたいなは accumulate




In [1]:
%%writefile reduce01.cpp
#include <algorithm>
#include <iostream>
#include <numeric>
#include <vector>
#define INF 100000000

std::vector<int> x = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
std::vector<int> y = {15, 12, 99, 27};
std::vector<int> z = {10, 20, 150, 100};
std::vector<int> w = {1, 3, 10, 100, -12, 2, 4};

int multiply(int x, int y) { return x * y; }

int gcd(int a, int b) {
    if (!b) return a;
    return gcd(b, a % b);
}

int lcd(int a, int b) { return a * b / gcd(a, b); }

int myMin(int a, int b) { return (a < b) ? a : b; }

int myMax(int a, int b) { return (a > b) ? a : b; }

int main() {
    std::cout << accumulate(x.begin(), x.end(), 0) << "\n";
    std::cout << accumulate(x.begin(), x.end(), 1, multiply) << "\n";
    std::cout << accumulate(y.begin(), y.end(), y[0], gcd) << "\n";
    std::cout << accumulate(z.begin(), z.end(), z[0], lcd) << "\n";
    std::cout << accumulate(w.begin(), w.end(), INF, myMin) << "\n";
    std::cout << accumulate(w.begin(), w.end(), -INF, myMax) << "\n";

    return 0;
}


Writing reduce01.cpp


In [2]:
!g++ reduce01.cpp -o reduce01; ./reduce01

55
3628800
3
300
-12
100


In [ ]:
%%writefile reduce02.cpp
#include <algorithm>
#include <iostream>
#include <numeric>
#include <vector>

int main(int argc, char **argv) {
    // add
    std::vector<int> a;

    for (int i = 0; i < 10; i++) a.push_back(i);

    std::cout << accumulate(a.begin(), a.end(), 1000) << "\n";  // 1045

    // string 
    std::vector<std::string> b;

    b.push_back("hello, ");
    b.push_back("world");
    b.push_back("!!");

    std::cout << accumulate(b.begin(), b.end(), std::string(""))
              << "\n";  // hello, world!!

    // binary operator
    std::cout << accumulate(a.begin(), a.end(), 1, [](int x, int y) {
        return std::max(x, y);
    }) << "\n";
    // 9

    return 0;
}

Overwriting reduce02.cpp


In [ ]:
!g++ reduce02.cpp -o reduce02; ./reduce02

1045
hello, world!!
9


# map 的なことは transform

In [ ]:
%%writefile map01.cpp
#include <algorithm>
#include <iostream>
#include <vector>

template <typename T>
void disp(const std::vector<T> &x) {
    for (size_t i = 0; i < x.size(); i++) std::cout << x[i] << " ";
    std::cout << "\n";
}

int main(int argc, char **argv) {
    std::vector<int> a;
    for (int i = 0; i < 10; i++) a.push_back(i);

    // 二乗を計算
    transform(a.begin(), a.end(), a.begin(), [](int x) { return x * x; });
    disp(a);  // 0 1 4 9 16 25 36 49 64 81

    // 入力と出力の型が違ってもOK
    std::vector<char> b;
    transform(a.begin(), a.end(), back_inserter(b),
              [](int x) { return 'A' + x % 26; });
    disp(b);  // A B E J Q Z K X M D

    // フィボナッチ数列
    std::vector<int> fib(20, 1);
//    transform(fib.begin(), fib.end() - 2, fib.begin() + 1, fib.begin() + 2,
//              [](int x, int y) { return x + y; });
    disp(fib);  // 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181
                // 6765

    return 0;
}

Overwriting map01.cpp


In [ ]:
!g++ map01.cpp -o map01; ./map01

0 1 4 9 16 25 36 49 64 81 
A B E J Q Z K X M D 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 


# filter は copy_if

In [ ]:
%%writefile filter01.cpp
#include <algorithm>
#include <iostream>
#include <vector>

template <typename T>
void disp(const std::vector<T> &x) {
    for (size_t i = 0; i < x.size(); i++) std::cout << x[i] << " ";
    std::cout << "\n";
}

int main(int argc, char **argv) {
    std::vector<int> a;
    for (int i = -5; i <= 5; i++) a.push_back(i);

    // 偶数のみ抽出
    std::vector<int> b;
    copy_if(a.begin(), a.end(), back_inserter(b),
            [](int x) { return x % 2 == 0; });
    disp(b);  // -4 -2 0 2 4

    // 負の数のみ抽出
    std::vector<int> c;
    copy_if(a.begin(), a.end(), back_inserter(c), [](int x) { return x < 0; });
    disp(c);  // -5 -4 -3 -2 -1

    return 0;
}

Overwriting filter01.cpp


In [ ]:
!g++ filter01.cpp -o filter01; ./filter01

-4 -2 0 2 4 
-5 -4 -3 -2 -1 
